## Import balíčků

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

## Načtení souborů

In [2]:
df_file_all = pd.read_csv('data/dc_file_modified2.csv')
df_evt_all = pd.read_csv('data/dc_evt_modified2.csv')

/tmp/ipykernel_38511/2963581421.py:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_file_all = pd.read_csv('data/dc_file_modified2.csv')
/tmp/ipykernel_38511/2963581421.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_evt_all = pd.read_csv('data/dc_evt_modified2.csv')


## Promazání zbytečných příznaků

příznaky jsme po jednom prošli, většinou měli příliš mnoho hodnot, byly duplicitní nebo nebyly podstatné pro naši analýzu.

In [3]:
df_file = df_file_all.drop(columns=['Unnamed: 0', 'id', 'filename', 'extra', 'datetime', 'file_path', 'name', 'typef', 'dir_type', 'file_type', 'is_allocated', 'is_allocated0', 'sha_256', 'epochtime', 'hour', 'minute'])
df_evt = df_evt_all.drop(columns=['filename_type', 'keywords'])

## Přetypování příznaků

In [4]:
bools = [
         'M', 'A', 'C', 'B',
         'file_stat', 'NTFS_file_stat', 'file_entry_shell_item', 'NTFS_USN_change',
         'filef', 'directory', 'link',
         'dir_appdata', 'dir_win', 'dir_user', 'dir_other',
         'file_executable', 'file_graphic', 'file_documents', 'file_ps', 'file_other',
         'mft', 'lnk_shell_items', 'olecf_olecf_automatic_destinations/lnk/shell_items', 'winreg_bagmru/shell_items', 'usnjrnl',
         'is_allocated1'
         ]

df_file['timestamp'] = df_file['timestamp'].astype('datetime64[ns]')

df_file[bools] = df_file[bools].astype('boolean')

# procento aktivit v mesici, kdy doslo k inidentu
# 100 * (df_file['timestamp'] >= '2020-09-01').sum() / df_file.shape[0]
dict(df_file.dtypes)

{'inode': dtype('int64'),
 'M': BooleanDtype,
 'A': BooleanDtype,
 'C': BooleanDtype,
 'B': BooleanDtype,
 'file_stat': BooleanDtype,
 'NTFS_file_stat': BooleanDtype,
 'file_entry_shell_item': BooleanDtype,
 'NTFS_USN_change': BooleanDtype,
 'filef': BooleanDtype,
 'directory': BooleanDtype,
 'link': BooleanDtype,
 'dir_appdata': BooleanDtype,
 'dir_win': BooleanDtype,
 'dir_user': BooleanDtype,
 'dir_other': BooleanDtype,
 'file_executable': BooleanDtype,
 'file_graphic': BooleanDtype,
 'file_documents': BooleanDtype,
 'file_ps': BooleanDtype,
 'file_other': BooleanDtype,
 'mft': BooleanDtype,
 'lnk_shell_items': BooleanDtype,
 'olecf_olecf_automatic_destinations/lnk/shell_items': BooleanDtype,
 'winreg_bagmru/shell_items': BooleanDtype,
 'usnjrnl': BooleanDtype,
 'is_allocated1': BooleanDtype,
 'file_size': dtype('int64'),
 'timestamp': dtype('<M8[ns]')}

### Příznak file_size

změníme file_size podle toho, do jakého intervalu spadá
```
0: 0
1: 1 - 1_000
2: 1_001 - 10_000
3: 10_001 - 100_000
4: 100_001 - 1_000_000
5: 1_000_001+
```

In [5]:
df_file['file_size+1_log'] = np.log(df_file['file_size'] + 1)

size_stamps = [-float('inf'), 0, 1_000, 10_000, 100_000, 1_000_000, float('inf')]
size_labels = ['0', '1-1_000', '1_001-10_000', '10_001-100_000', '100_001-1_000_000', '1_000_000+']
# make data ordinal based on size

df_file['file_size'] = pd.cut(df_file['file_size'], size_stamps, labels=False)
df_file['file_size'].value_counts()

0    572239
2     85508
1     82970
3     66466
4     30209
5      6471
Name: file_size, dtype: int64

In [6]:
df_file.sample(10)

,inode,M,A,C,B,file_stat,NTFS_file_stat,file_entry_shell_item,NTFS_USN_change,filef,...,file_other,mft,lnk_shell_items,olecf_olecf_automatic_destinations/lnk/shell_items,winreg_bagmru/shell_items,usnjrnl,is_allocated1,file_size,timestamp,file_size+1_log
374189,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-17 15:52:04,0.000000
799244,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-17 17:50:39,0.000000
816880,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-17 17:51:35,0.000000
51051,36048,False,True,False,True,True,False,False,False,True,...,True,False,False,False,False,False,True,2,2013-08-22 04:15:12,8.877521
267272,0,True,True,False,False,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2014-03-21 18:07:06,0.000000
2454,34418,True,False,False,False,True,False,False,False,True,...,True,False,False,False,False,False,True,1,2013-06-18 12:31:07,6.837333
693943,0,False,False,True,False,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2020-09-17 16:50:21,0.000000
447276,22020,False,False,True,False,True,False,False,False,True,...,True,False,False,False,False,False,True,3,2020-09-17 16:48:27,10.116379
706932,0,False,False,True,False,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2020-09-17 16:50:29,0.000000
171057,0,True,True,False,False,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2013-08-22 15:23:27,0.000000


In [7]:
#? tohle by bylo nice, ale je to moc pomale
# def find_minute_frequency(row):
#     return df_file[(df_file['timestamp'] >= row['timestamp'] - pd.Timedelta(minutes=1)) \
                #    & (df_file['timestamp'] < row['timestamp'] + pd.Timedelta(minutes=1))].shape[0]

# df_file.apply(find_minute_frequency, axis=1)

In [8]:
floored_to_min = df_file['timestamp'].dt.floor('min')
counts = floored_to_min.value_counts().to_dict()

# minute_activity = df_file['timestamp'].apply(lambda x: counts[x.floor('min')])
# minute_activity.to_csv('data/minute_activity.csv')
minute_activity = pd.read_csv('data/minute_activity.csv', index_col=0)['timestamp']

In [9]:
df_file['minute_activity'] = minute_activity

In [10]:
inode_counts = df_file['inode'].value_counts().to_dict()

inode_activity = df_file['inode'].apply(lambda x: inode_counts[x])

In [11]:
df_file['inode_activity'] = inode_activity
df_file.drop(columns=['inode', 'timestamp'], inplace=True)
df_file['epochtime'] = df_file_all['epochtime'].astype('int64')

In [12]:
df_file.to_csv('data/dc_file_preprocessed.csv', index=False)

## Standardizace

In [21]:
standard_scaler = StandardScaler()
numeric = df_file.drop(columns='timestamp')

In [22]:
df_file_standardized = pd.DataFrame(standard_scaler.fit_transform(numeric), columns=numeric.columns)

In [26]:
df_file_standardized.to_csv('data/dc_file_preprocessed_standardized.csv', index=False)